# Лабораторная работа №4

## Подготовка входных данных

In [30]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [31]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

In [32]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

In [34]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})
X

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
113780,0,13.2,25.7,0.0,0,39.0,0,0,15.0,24.0,72.0,57.0,1020.1,1017.3,18.8,23.2,False,False
38968,1,14.9,31.4,0.0,1,33.0,1,1,22.0,9.0,42.0,22.0,1024.5,1019.8,21.0,29.5,False,False
62045,2,2.3,14.1,0.4,2,17.0,2,0,9.0,13.0,99.0,72.0,1029.8,1027.3,5.5,12.8,False,False
134750,3,11.4,18.0,9.2,3,52.0,3,2,20.0,15.0,84.0,65.0,1010.1,1007.1,14.4,16.6,True,True
143182,4,22.6,33.6,0.0,2,46.0,0,1,17.0,26.0,53.0,40.0,1020.1,1015.8,26.2,33.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42745,18,11.4,15.6,4.6,0,41.0,11,12,19.0,19.0,74.0,88.0,1028.0,1027.6,13.1,14.2,True,True
20203,15,19.3,23.5,3.0,15,43.0,13,13,28.0,22.0,94.0,86.0,1014.8,1013.2,19.6,21.7,True,True
138845,10,9.7,24.9,0.0,0,37.0,8,12,22.0,13.0,10.0,7.0,1017.6,1013.6,17.5,23.2,False,False
38340,1,13.2,20.9,1.4,8,48.0,10,5,28.0,26.0,62.0,35.0,1007.9,1008.1,15.2,18.7,True,False


In [37]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

In [40]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

Провести классификацию найденного датасета, методами наивного Байеса. В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [55]:
import numpy as np

import sklearn.naive_bayes

### Гауссовский наивный Байес

In [58]:
gaussian_classifier = sklearn.naive_bayes.GaussianNB()
gaussian_params ={
    'var_smoothing': np.geomspace(1e-10,1e10,num=100)
}
gaussian_grid = GridSearchCV(gaussian_classifier, gaussian_params, cv=2, n_jobs=-1,verbose=3)
gaussian_grid.fit(X_train, y_train)
best_gaussian_params = gaussian_grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


In [59]:
best_gaussian_model = sklearn.naive_bayes.GaussianNB(**best_gaussian_params)
best_gaussian_model.fit(X_train, y_train)
gaussian_predicted = best_gaussian_model.predict(X_test)
print('Used params:',best_gaussian_params)
print('Evaluation:\n', metrics.classification_report(y_test, gaussian_predicted,digits=5))

Used params: {'var_smoothing': 0.7924828983539186}
Evaluation:
               precision    recall  f1-score   support

       False    0.81867   0.98110   0.89255      9047
        True    0.78354   0.23946   0.36681      2585

    accuracy                        0.81628     11632
   macro avg    0.80111   0.61028   0.62968     11632
weighted avg    0.81086   0.81628   0.77572     11632



### Мультиномиальный наивный Байес

In [61]:
classifier = sklearn.naive_bayes.MultinomialNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(X_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


In [62]:
best_model = sklearn.naive_bayes.MultinomialNB(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Used params: {'alpha': 1e-10}
Evaluation:
               precision    recall  f1-score   support

       False    0.77932   0.99967   0.87585      9047
        True    0.88889   0.00928   0.01838      2585

    accuracy                        0.77957     11632
   macro avg    0.83410   0.50448   0.44711     11632
weighted avg    0.80367   0.77957   0.68529     11632



### Комплементарный наивный Байес

In [64]:
classifier = sklearn.naive_bayes.ComplementNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(X_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


In [65]:
best_model = sklearn.naive_bayes.ComplementNB(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Used params: {'alpha': 8697.490026177835}
Evaluation:
               precision    recall  f1-score   support

       False    0.82729   0.91279   0.86794      9047
        True    0.52182   0.33308   0.40661      2585

    accuracy                        0.78396     11632
   macro avg    0.67455   0.62293   0.63727     11632
weighted avg    0.75940   0.78396   0.76542     11632



### Бернуллиевский наивный Байес

In [67]:
classifier = sklearn.naive_bayes.BernoulliNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100)
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(X_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


In [70]:
best_model = sklearn.naive_bayes.BernoulliNB(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Used params: {'alpha': 1353.0477745798075}
Evaluation:
               precision    recall  f1-score   support

       False    0.77796   0.99878   0.87465      9047
        True    0.35294   0.00232   0.00461      2585

    accuracy                        0.77734     11632
   macro avg    0.56545   0.50055   0.43963     11632
weighted avg    0.68351   0.77734   0.68130     11632



### Категориальный наивный Байес

In [74]:
classifier = sklearn.naive_bayes.CategoricalNB()
params ={
    'alpha': np.geomspace(1e-10,1e10,num=100),
    'min_categories':[2]
}
grid = GridSearchCV(classifier, params, cv=2, n_jobs=-1,verbose=3)
grid.fit(X_train, y_train)
best_params = grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


In [76]:
best_model = sklearn.naive_bayes.CategoricalNB(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))

Used params: {'alpha': 1e-10, 'min_categories': 2}
Evaluation:
               precision    recall  f1-score   support

       False    0.78360   0.99259   0.87580      9047
        True    0.61047   0.04062   0.07617      2585

    accuracy                        0.78104     11632
   macro avg    0.69703   0.51661   0.47598     11632
weighted avg    0.74512   0.78104   0.69810     11632

